In [1]:
import overpass
api = overpass.API(timeout=1000)

import requests

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][timeout:1000];
(
node(41.9023,-83.1270,42.3452,-82.4810);
way(41.9023,-83.1270,42.3452,-82.4810);
relation(41.9023,-83.1270,42.3452,-82.4810);
);
out geom;
"""

response = requests.get(overpass_url, 
                        params={'data': overpass_query})
response.raise_for_status()
data = response.json()

In [21]:
from tqdm import tqdm
import pandas as pd
from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster

import osmnx as ox
G = ox.graph_from_bbox(42.3567,41.9023,-82.4810,-83.1270, network_type='drive')

import networkx as nx

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [5]:
df = pd.DataFrame(data['elements'])

In [6]:
node = df[df['type'] == 'node'][['id','lat','lon','tags']].reset_index(drop=True)
way = df[df['type'] == 'way'][['id','tags','nodes','geometry','bounds']].reset_index(drop=True)
way = pd.concat([way, pd.DataFrame(way['bounds'].apply(pd.Series))], axis=1).drop(['bounds'] , axis=1)
relation = df[df['type'] == 'relation'][['id','tags','bounds','members']].reset_index(drop=True)

In [7]:
way = way[~way.tags.astype(str).str.contains('highway')].reset_index(drop=True)
way = way[~way.tags.astype(str).str.contains('building')].reset_index(drop=True)

way['lat'] = way.geometry.apply(lambda x: x[0].get('lat'))
way['lon'] = way.geometry.apply(lambda x: x[0].get('lon'))

In [8]:
node.head()

,id,lat,lon,tags
0,11753030,42.254737,-83.017250,{'highway': 'traffic_signals'}
1,11753034,42.253266,-83.016344,NaN
2,11753047,42.250849,-83.014818,NaN
3,11753061,42.245324,-83.011312,NaN
4,11753066,42.243022,-83.009830,NaN


In [9]:
way.head()

,id,tags,nodes,geometry,minlat,minlon,maxlat,maxlon,lat,lon
0,3772706,NaN,"[18826703, 6555840684, 6555840685, 6555840686,...","[{'lat': 42.3426353, 'lon': -82.9274882}, {'la...",42.342630,-82.938412,42.348977,-82.920663,42.342635,-82.927488
1,3773233,{'source': 'PGS'},"[18830711, 18830713, 18830716, 18830718, 18830...","[{'lat': 42.3321669, 'lon': -82.9715237}, {'la...",42.322686,-83.030173,42.332312,-82.971524,42.332167,-82.971524
2,3773625,{'source': 'Yahoo'},"[18833654, 18833668, 18833681, 7407088295, 740...","[{'lat': 42.3045256, 'lon': -83.1598057}, {'la...",42.279816,-83.159806,42.304526,-83.118429,42.304526,-83.159806
3,3773714,{'source': 'Yahoo'},"[18834421, 5534434161, 5534434162, 7667629126,...","[{'lat': 42.2879503, 'lon': -83.1201973}, {'la...",42.287950,-83.120273,42.290403,-83.102992,42.287950,-83.120197
4,3773735,{'source': 'Yahoo Imaging'},"[5534870795, 5534870793, 5534870797, 553487079...","[{'lat': 42.2949667, 'lon': -83.096408}, {'lat...",42.294967,-83.096408,42.314086,-83.075870,42.294967,-83.096408


In [10]:
relation.head()

,id,tags,bounds,members
0,68841,"{'ISO3166-2': 'CA-ON', 'admin_level': '4', 'bo...","{'minlat': 41.6765556, 'minlon': -95.1537399, ...","[{'type': 'node', 'ref': 305700702, 'role': 'l..."
1,104454,"{'is_in:state': 'MI', 'name': 'I 94 (MI)', 'ne...","{'minlat': 41.7600376, 'minlon': -86.7512221, ...","[{'type': 'way', 'ref': 365412463, 'role': 'we..."
2,134591,"{'admin_level': '8', 'border_type': 'city', 'b...","{'minlat': 42.255192, 'minlon': -83.287959, 'm...","[{'type': 'node', 'ref': 18994998, 'role': 'la..."
3,134592,"{'admin_level': '8', 'border_type': 'city', 'b...","{'minlat': 42.2571832, 'minlon': -83.1531187, ...","[{'type': 'way', 'ref': 34365867, 'role': 'out..."
4,134593,"{'admin_level': '8', 'border_type': 'city', 'b...","{'minlat': 42.23422, 'minlon': -83.161635, 'ma...","[{'type': 'way', 'ref': 34365869, 'role': 'out..."


In [11]:
cs = pd.read_csv('ev_charging_station.csv')
cs.head()

,lat,lon,Plugtype,Level,Plugs,Access,Status
0,42.317491,-83.044508,EV Plug (J1772),2,2,"Network App, Network RFID, Phone",Existing
1,42.245872,-83.060583,EV Plug (J1772),2,6,NaN,Existing
2,42.273509,-83.048010,EV Plug (J1772),2,1,NaN,Existing
3,42.283664,-83.017812,EV Plug (J1772),2,1,NaN,Existing
4,42.292550,-83.012565,EV Plug (J1772),2,1,NaN,Existing


In [ ]:
# distance = []

# G = ox.graph_from_bbox(42.3567,41.9023,-82.4810,-83.1270, network_type='drive')

# for i in tqdm(range(len(way))):
#     origin_node = ox.distance.nearest_nodes(G, way.lon[i], way.lat[i])
    
#     bbox = ox.utils_geo.bbox_from_point((way.lat[i], way.lon[i]), dist=5000)
    
#     poly = ox.utils_geo.bbox_to_poly(bbox[0],bbox[1],bbox[2],bbox[3])
#     dist = []
    
#     for j in range(len(cs)):     
        
#         point = Point(cs.lon[j], cs.lat[j])
        
#         if point.within(poly):
#             destination_node = ox.distance.nearest_nodes(G, cs.lon[j], cs.lat[j])
            
#             try:
#                 distance_in_kilometers =  nx.shortest_path_length(G, origin_node, destination_node, weight='length') / 1000
#                 dist.append([j,distance_in_kilometers])
#             except:
#                 distance_in_kilometers = 0
    
#     distance.append(dist)
    
# way['distance'] = distance
# way['num_charging_station'] = way.distance.apply(lambda x: len(x))
# way['max_distance'] = way.distance.apply(lambda x: max(x ,key= lambda x: x[1])[1] if len(x)>0 else -1)
# way['min_distance'] = way.distance.apply(lambda x: min(x ,key= lambda x: x[1])[1] if len(x)>0 else -1)

In [13]:
distance = []
for i in tqdm(range(len(cs))):
    origin_node = ox.distance.nearest_nodes(G, cs.lon[i], cs.lat[i])

    dist = []
    
    for j in range(i+1,len(cs)):     
        
        destination_node = ox.distance.nearest_nodes(G, cs.lon[j], cs.lat[j])
        
        distance_in_kilometers =  nx.shortest_path_length(G, origin_node, destination_node, weight='length') / 1000

        distance.append([i,j,distance_in_kilometers])

100%|██████████| 35/35 [01:11<00:00,  2.05s/it]


In [14]:
dist = pd.DataFrame(distance,columns=['st1','st2','distance'])
dist.sort_values(by=['distance'],ascending=False,inplace=True)
dist.reset_index(drop = True,inplace=True)

In [24]:
dist.to_csv('distance_between_charging_stations.csv')

In [17]:
n = 4

loc = []
address = []
c,i = 0,0

while True:
    # print(c)
    lat = (cs.iloc[dist.st1.iloc[i]].lat + cs.iloc[dist.st2.iloc[i]].lat )/2
    lon = (cs.iloc[dist.st1.iloc[i]].lon + cs.iloc[dist.st2.iloc[i]].lon )/2
    i+=1
    
    nearest_node = ox.distance.nearest_nodes(G, lon, lat)
    lat = G.nodes[nearest_node].get('y')
    lon = G.nodes[nearest_node].get('x')
    tmp = [lat,lon]
    
    if tmp in loc:
        continue
    d = []
    for j in range(len(cs)):
        destination_node = ox.distance.nearest_nodes(G, cs.lon[j], cs.lat[j])
        distance_in_kilometers =  nx.shortest_path_length(G, nearest_node, destination_node, weight='length') / 1000
        d.append(distance_in_kilometers)
        
    if min(d)<5:
        continue
    # print(c,n) 
    loc.append([lat,lon])
    address.append(geolocator.reverse(str(lat)+','+str(lon)).raw.get('address'))    
    c+=1
    if c==n:
        break

In [18]:
address = pd.concat([pd.json_normalize(address),pd.DataFrame(loc,columns=['lat','lon'])],axis = 1)

In [19]:
address['Status'] = 'New'

In [20]:
df = cs.append(address[['lat','lon','Status']]).reset_index(drop=True)

In [27]:
m=folium.Map(
    location=[df['lat'].mean(), df['lon'].mean()],
    zoom_start=10,)

cluster = MarkerCluster(name="Electric Vehicle Charging Stations", options={"showCoverageOnHover": False})

def get_icon(status):
    if status == "Existing":
        return folium.Icon(icon="bolt", prefix='fa',
                       color='black',
                       icon_color='#2ecc71'
                       )
    else:
        return folium.Icon(icon="glyphicon-time",
                       color='black',icon_color='yellow')
df.apply(
    lambda row: folium.Marker(
        location=[row['lat'], row['lon']],
        popup=[row['lat'], row['lon']],
        tooltip=row['Plugtype'],
        icon=get_icon(row['Status']),
        ).add_to(cluster),
    axis=1)
cluster.add_to(m)
sw = df[['lat', 'lon']].min().values.tolist()
ne = df[['lat', 'lon']].max().values.tolist()

m.fit_bounds([sw,ne]) 
m.add_child(folium.LatLngPopup())
m